In [1]:
'''
!pip install kafka-python
!pip install pyspark
!curl -sSOL https://dlcdn.apache.org/kafka/3.1.0/kafka_2.13-3.1.0.tgz
!tar -xzf kafka_2.13-3.1.0.tgz
!./kafka_2.13-3.1.0/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.1.0/config/zookeeper.properties
!./kafka_2.13-3.1.0/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.1.0/config/server.properties
!sleep 10
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --delete --topic input_topic --bootstrap-server localhost:9092
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --topic input_topic --bootstrap-server localhost:9092 --partitions 3 --replication-factor 1
!./kafka_2.13-3.1.0/bin/kafka-topics.sh --create --topic output_topic --bootstrap-server localhost:9092 --partitions 3 --replication-factor 1
!./kafka_2.13-3.1.0/bin/kafka-console-consumer.sh  --bootstrap-server localhost:9092 --topic input_topic --from-beginning
!rm iris.csv
!wget https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv
!ls
'''

import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import pyspark
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import math
import string
import random
from datetime import datetime
import time
from json import dumps
import random
import csv
import pandas as pd
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

conf = pyspark.SparkConf().set('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0')
sc = pyspark.SparkContext(conf = conf)
ssc = StreamingContext(sc,1)
KAFKA_INPUT = "input_topic"
KAFKA_OUTPUT = "output_topic"
KAFKA_SERVER = "127.0.0.1:9092"
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .getOrCreate()

data=spark.read.format("csv").option("header","true").option("delimiter",",").load("iris.csv")
original_schema = data.schema
feature_cols = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
data = data.select(*(col(c).cast("double") for c in feature_cols), "species")
data = assembler.transform(data)
data = data.select(['features', 'species'])
label_indexer = StringIndexer(inputCol='species', outputCol='label').fit(data)
data = label_indexer.transform(data)
data = data.select(['features', 'label'])

reg = 0.01
train, test = data.randomSplit([0.70, 0.30])
lr = LogisticRegression(regParam=reg)
model = lr.fit(train)
prediction = model.transform(test)

print("Prediction")
prediction.show(10)
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = evaluator.evaluate(prediction)

print('Accuracy', accuracy)

Ivy Default Cache set to: /Users/gui_r/.ivy2/cache
The jars for the packages stored in: /Users/gui_r/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-292d4a96-f6ff-4976-beac-2062eeaa255e;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/Caskroom/miniconda/base/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 255ms :: artifacts dl 5ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-log

Prediction
+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[4.3,3.0,1.1,0.1]|  0.0|[6.54087870810077...|[0.98343057765760...|       0.0|
|[4.6,3.2,1.4,0.2]|  0.0|[6.11478298858913...|[0.97829709218680...|       0.0|
|[4.6,3.4,1.4,0.3]|  0.0|[6.35512818379031...|[0.98714780682076...|       0.0|
|[4.7,3.2,1.6,0.2]|  0.0|[5.79309209783107...|[0.96929858420403...|       0.0|
|[4.8,3.0,1.4,0.1]|  0.0|[5.62923168862030...|[0.94834890280808...|       0.0|
|[4.9,2.5,4.5,1.7]|  2.0|[-2.1028499239658...|[0.02048751526399...|       1.0|
|[4.9,3.0,1.4,0.2]|  0.0|[5.29458787529601...|[0.93189909625800...|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|[5.63327769512019...|[0.95087485767876...|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|[5.63327769512019...|[0.95087485767876...|       0.0|
|[5.0,3.2,1.2,0.2]|  0.0|[5.8234586174280

In [2]:
df2 = stream_df.select( from_json(col("value").cast("string"), original_schema).alias("values")).select("values.*")

NameError: name 'stream_df' is not defined

In [3]:
stream_df = spark \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", KAFKA_SERVER) \
            .option("subscribe", KAFKA_INPUT) \
            .option("startingOffsets", "latest") \
            .load()

'''query = (stream_df.writeStream
                  .outputMode("append")
                  .format("memory")
                  .queryName("kafka")
                  .start())'''


#df = spark.table("kafka")
#df2 = df.select( from_json(col("value").cast("string"), original_schema).alias("values")).select("values.*")
df2 = stream_df.select( from_json(col("value").cast("string"), original_schema).alias("values")).select("values.*")

data = df2
feature_cols = data.columns[:-1]
data = data.select(*(col(c).cast("double") for c in feature_cols), "species")
data = assembler.transform(data)
data = data.select(['features', 'species'])
data = label_indexer.transform(data)
data = data.select(['features', 'label'])

prediction = model.transform(data)

'''print("Prediction")
prediction.show(10)

evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = evaluator.evaluate(prediction)
print('Accuracy', accuracy)
'''


'print("Prediction")\nprediction.show(10)\n\nevaluator = MulticlassClassificationEvaluator(metricName=\'accuracy\')\naccuracy = evaluator.evaluate(prediction)\nprint(\'Accuracy\', accuracy)\n'

In [4]:
prediction = prediction.select("label", "prediction")

In [ ]:
prediction \
  .select(to_json(struct([col(c).alias(c) for c in prediction.columns])).alias("value"))\
  .writeStream \
  .format("kafka") \
  .option("checkpointLocation", "/tmp/") \
  .option("kafka.bootstrap.servers", KAFKA_SERVER) \
  .option("topic", KAFKA_OUTPUT) \
  .start() \
  .awaitTermination()

22/05/06 12:59:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
